In [1]:
import sklearn
from sklearn import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
import torch.optim as optim 
import sys 
import os 
import matplotlib.pyplot as plt

In [2]:
project_basedir=os.path.join('..','..','..')
project_basedir = os.path.abspath(project_basedir)
sys.path.insert(0,project_basedir)

In [3]:
from core.np.datasets.IrisDataset import Iris 
import core.np.Nodes as node 
from core import debug, info, log_at_info
import core.np.Nodes as node
import core.np.Loss as loss
import core.np.Activations as act
from  core.np.utils import to_one_hot

In [4]:
iris = Iris()
learning_rate = 0.01 

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dense =  nn.Linear(4,3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dense(x) 
        x = self.softmax(x)
        return x 

In [6]:
input = torch.FloatTensor([[1, -1, 2,1]])
print(input)
net = Net() 
output = net(input)
print(output)

tensor([[ 1., -1.,  2.,  1.]])
tensor([[0.3310, 0.5128, 0.1562]], grad_fn=<SoftmaxBackward>)


In [7]:
target = torch.tensor([1])
target = target.view(1, )
criteria = nn.NLLLoss()
loss = criteria(output, target)
optimizer = torch.optim.SGD(net.parameters(), lr=.01)

In [22]:
epoch = 0 
log_at_info() 
for x, y in iris.train_iterator(1,batch_size=1, one_hot=False): 
    torch_x = torch.from_numpy(x.T).float() 
    target_y = torch.from_numpy(y).long()  
    print("x={}, y={}".format(torch_x, target_y.data))
    predicted = net(torch_x) 
    loss = criteria(predicted, target_y )
    if epoch %1000==0 : 
        print("[{}] Loss:{}".format(epoch, loss))
    epoch+=1
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 

x=tensor([[5.1000, 3.5000, 1.4000, 0.2000]]), y=tensor([1])
[0] Loss:-0.9835102558135986


In [28]:
num_correct = 0 
test_indexes = list(iris.test_idx)
for i in test_indexes: 
    x = torch.from_numpy(iris.data[i,:].reshape(-1,4)).float() 
    y_actual = iris.targets[i] 
    y_pred = net(x) 
    max_idx = torch.argmax(y_pred) 
    mark = "x"
    if max_idx.data == y_actual : 
        mark = u'\u2713'
        num_correct+=1
    print("X:{}, Target={}, Predicted:{}  {}".format( x, y_actual, max_idx, mark))
    #print(repr(value))

        
total = len(test_indexes)
percent = num_correct*100.0/total
print(percent)

X:tensor([[4.9000, 3.0000, 1.4000, 0.2000]]), Target=0, Predicted:1  x
X:tensor([[7.9000, 3.8000, 6.4000, 2.0000]]), Target=2, Predicted:1  x
X:tensor([[4.6000, 3.4000, 1.4000, 0.3000]]), Target=0, Predicted:1  x
X:tensor([[5.4000, 3.7000, 1.5000, 0.2000]]), Target=0, Predicted:1  x
X:tensor([[4.3000, 3.0000, 1.1000, 0.1000]]), Target=0, Predicted:1  x
X:tensor([[5.8000, 4.0000, 1.2000, 0.2000]]), Target=0, Predicted:1  x
X:tensor([[5.1000, 3.5000, 1.4000, 0.3000]]), Target=0, Predicted:1  x
X:tensor([[5.7000, 3.8000, 1.7000, 0.3000]]), Target=0, Predicted:1  x
X:tensor([[6.7000, 3.0000, 5.2000, 2.3000]]), Target=2, Predicted:1  x
X:tensor([[6.3000, 2.5000, 5.0000, 1.9000]]), Target=2, Predicted:1  x
X:tensor([[5.1000, 3.7000, 1.5000, 0.4000]]), Target=0, Predicted:1  x
X:tensor([[6.5000, 3.0000, 5.2000, 2.0000]]), Target=2, Predicted:1  x
X:tensor([[6.2000, 3.4000, 5.4000, 2.3000]]), Target=2, Predicted:1  x
X:tensor([[4.8000, 3.4000, 1.9000, 0.2000]]), Target=0, Predicted:1  x
X:tens